# Part 1

In [1]:
source = open("05.txt", "r").read()
sections = source.split("\n\n")

seed_list = sections[0];
seeds = seed_list.split(" ")[1:]
maps = sections[1:]

location_maps = []
for map in maps:
    lines = map.split("\n")
    label = lines[0]
    mappings = lines[1:]
    location_map = {"label": label, "mappings": []}

    for line in mappings:
        target_start, seed_start, seed_range = line.split(" ")
        seed_start = int(seed_start)
        target_start = int(target_start)
        seed_range = int(seed_range)
        location_map["mappings"].append((seed_start, target_start, seed_range))
    location_map["mappings"] = sorted(location_map["mappings"], key=lambda x: x[0])
    location_maps.append(location_map)
def apply_map(start, location_map):
    for mapping in location_map["mappings"]:
        seed_start, target_start, seed_range = mapping
        if start >= seed_start and start < seed_start + seed_range:
            return target_start + (start - seed_start)
    return start
ends = []
for seed in seeds:
    start = int(seed)
    for location_map in location_maps:
        start = apply_map(start, location_map)
    ends.append(start)
ends
min(ends)

993500720

# Part 2

A windowing approach to look only at min/maxes

In [11]:
import numpy as np 
seed_ranges = (np.array_split([int(x) for x in seeds], len(seeds)/2))
iterations = 0
for location_map in location_maps:
    windows = []
    for seed_start, seed_range in seed_ranges:
        #evaluate if seed range is in location range
        seed_max = seed_start + seed_range
        found = False
        do_loop = True
        while do_loop == True: 
            seed_max = seed_start + seed_range
            for location_seed_start, location_start, location_range in location_map["mappings"]:
                location_seed_max = location_seed_start + location_range
                location_offset = location_start - location_seed_start
                #window possibilities
                if( seed_start < location_seed_start):
                    if(seed_max < location_seed_start):
                        do_loop = False
                        found = True
                        break;
                    else:
                        windows.append((seed_start, location_seed_start - 1))
                        seed_start = location_seed_start
                        seed_range = seed_max - location_seed_start
                        found = True
                if(seed_start >= location_seed_start and seed_start < location_seed_max):
                    # Starts within
                    if(seed_max <= location_seed_max):
                        # within
                        windows.append((seed_start + location_offset, seed_range))
                        do_loop = False
                        found = True
                        break;
                    else:
                        windows.append((seed_start + location_offset, location_seed_max - seed_start))
                        seed_start = location_seed_max
                        seed_range = seed_max - location_seed_max
                        seed_max = seed_start + seed_range
                        # ends after
                if(seed_start >= location_seed_max): 
                     found = False
                if found == True : break
            if found == False:
                windows.append((seed_start, seed_range)) # If I miss all windows entirely that means theres no change to be had
                do_loop = False
    seed_ranges = list(set(windows))
new_seeds = np.array([(x[0], x[0] + x[1] -1 ) for x in seed_ranges if x[1] > 0]).flatten()
print("number of new seeds based on windowing", len(new_seeds))
min(new_seeds)

number of new seeds based on windowing 244


4917124